In [1]:
test_set_path = "../dataset/test-00000-of-00001.parquet"

In [2]:
import pandas as pd
test_df = pd.read_parquet(test_set_path)

In [3]:
from prompt.prompt import get_sentiment_classifier_prompt
from llm_service.llm_caller import LLMCaller
from config import LLMConfig, load_yaml_config
from utils.load_label_maps import load_label_maps
from utils.evaluate import get_accuracy
from utils.preprocess import *

import time
from tqdm import tqdm

In [4]:
def preprocess_pipelines(text: str) -> str:
    text = remove_url(text)
    text = remove_special_char(text)
    return text

In [5]:
LABEL_MAPS = load_label_maps("../dataset/label_maps.yaml")

In [6]:
load_yaml_config("../config/gemini_config.yaml")

In [7]:
llm_config = LLMConfig()
llm_service = LLMCaller(llm_config)

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
llm_config.MODEL

'gemini-1.5-flash'

In [13]:
import os

total_row = len(test_df)
ans_df = pd.DataFrame(columns=["texts", "category", "process_time"])

RESULTS_DIR = "../results/"
results_path = os.path.join(RESULTS_DIR, f"{llm_service.llm_config.MODEL}_preprocess_results_1.csv")

if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

for row in tqdm(range(0, 275)):
    sentence = test_df.loc[row, "texts"]
    preprocessed_sentence = preprocess_pipelines(sentence)
    sentiment_classifier_prompt = get_sentiment_classifier_prompt(message=preprocessed_sentence)
    start_process = time.time()
    ans = llm_service.call(sentiment_classifier_prompt)
    process_time = time.time() - start_process
    try:
        category = LABEL_MAPS[ans]
    except KeyError:
        category = ans

    ans_df.loc[row, "texts"] = sentence
    ans_df.loc[row, "category"] = category
    ans_df.loc[row, "process_time"] = process_time

    ans_df.to_csv(results_path, index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 275/275 [04:23<00:00,  1.05it/s]


In [14]:
ans_df = pd.read_csv("../results/gemini-1.5-flash_preprocess_results.csv")

In [10]:
acc = get_accuracy(test_df, ans_df)

ValueError: 2 dataframes not same length.

In [12]:
len(test_df)

2671

In [ ]:
acc

In [16]:
post_process_ans_df = pd.DataFrame(columns=["texts", "category", "process_time"])
for row in range(len(ans_df)):
    category = ans_df.loc[row, "category"]
    if isinstance(category, str):
        if "neutral" in category:
            category = LABEL_MAPS["neutral"]
        elif "positive" in category:
            category = LABEL_MAPS["positive"]
        elif "negative" in category:
            category = LABEL_MAPS["negative"]
        elif "question" in category:
            category = LABEL_MAPS["question"]

    post_process_ans_df.loc[row, "texts"] = ans_df.loc[row, "texts"]
    post_process_ans_df.loc[row, "category"] = category
    post_process_ans_df.loc[row, "process_time"] = ans_df.loc[row, "process_time"]

In [ ]:
acc = get_accuracy(test_df, post_process_ans_df)

In [ ]:
acc

In [ ]:
for row in range(len(post_process_ans_df)):
    pred = post_process_ans_df.loc[row, "category"]
    actual = test_df.loc[row, "category"]
    text = post_process_ans_df.loc[row, "texts"]
    if pred != actual:
        print(f"{text}, actual: {actual}, pred: {pred}")